<a href="https://colab.research.google.com/github/simonefmr/teste/blob/main/ranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www3.bcb.gov.br/ranking/historico.do

# **Pós Graduação Ciência de Dados - PUC Rio**
# **MVP Sprint I: Análise de Dados e Boas Práticas**
## Aluna: Simone de Fátima Marques Ramos

## **1. Introdução**

O  Ranking Bacen é formado a partir das reclamações do público registradas nos canais de atendimento do Banco Central (BC): internet, correspondência, presencialmente ou telefone (145). As demandas são analisadas de forma amostral pelo BC.

Participam do Ranking: bancos (comerciais, múltiplos, cooperativos, de investimentos), sociedades de crédito, financiamento e investimento (SCFI), instituições de pagamento e administradoras de consórcio.

São divulgados três arquivos, no formato CSV, com os seguintes detalhamentos:

I- Bancos e financeiras - Reclamações e quantidades de clientes por instituição financeira:
Apresenta as quantidades de reclamações e clientes, por instituição financeira, referentes ao mês
selecionado.

II- Bancos e financeiras - Irregularidades por instituição financeira:
Apresenta as quantidades de reclamações, por irregularidade e instituição financeira, referentes ao
mês selecionado.

III- Bancos e financeiras - Tabela de irregularidades:
Apresenta a tabela de irregularidades referente ao mês selecionado.

Neste projeto, irei abordar apenas os dados das Irregularidades por instituição financeira (II).

### **1.1. Problema de aprendizado supervisionado ou não supervisionado e Hipóteses sobre o problema**

Fazem parte do ranking as reclamações que foram analisadas e encerradas como:

Reclamações reguladas procedentes: casos em que houve sinal de descumprimento de lei ou norma pela instituição financeira no período indicado.
Reclamações reguladas – outras: casos em que não houve sinal de descumprimento de lei ou norma pela instituição financeira no período indicado (inclui as reclamações encerradas como improcedentes e não conclusivas).
Reclamações não reguladas: casos que não são tratados por lei ou norma expedida pelo Banco Central e não são sujeitos a sua supervisão.
Total de reclamações: total de reclamações reguladas procedentes, reclamações reguladas – outras e reclamações não reguladas.

Neste projeto analizarei apenas o **primeiro trimestre de 2023**. Sem base histórica, trataremos como um problema de **aprendizado não supervisionado**, sendo o objetivo entender as possíveis relações entre os dados apresentados.

### **1.2. Definição do Problema** 


O alto índice de reclamações ou reincidência das mesmas, podem levar o Banco Central a multar as instituições financeiras ou aplicar outras ferramentas de supervisão, como termos de compromisso, acordos de leniência, e medidas coercitivas e acautelatórias.

As instituições são classificadas pela ordem decrescente do índice de reclamações, ou seja, da mais reclamada para a menos reclamada.

As Reclamações reguladas – outras e as Reclamações não reguladas são divulgadas, porém não influenciam na posição das instituições no ranking.

No processo de análise, pode ser identificada mais de uma irregularidade para uma mesma reclamação. Nesse caso, cada irregularidade conta como uma ocorrência para cálculo do Ranking de Reclamações.

Tentar entender as origens das reclamações é essencial para monitoramento do correto funcionamento do sistema financeiro pelo Bacen, assim como pelas instituições financeiras, que buscam atendimento de qualidade e não desejam ter suas atividades limitadas ou restritas por força de sanção.

## **2. Importação de pacotes e bibliotecas**

In [2]:
# Importando bibliotecas e pacotes que serão utilizados na análise

# Importação de Bibliotecas
import pandas as pd
import numpy as np
import glob
import seaborn as sns
import statistics
import matplotlib.pyplot as plt
from matplotlib import cm
import plotly.express as px
from pandas import set_option
from pandas.plotting import scatter_matrix

from IPython.utils.sysinfo import encoding

# para tratamento de data
import datetime

# para tratamento de missings
import missingno as ms 

# Configuração para não exibir os warnings
import warnings
warnings.filterwarnings("ignore")

## transformações numéricas
from sklearn.preprocessing import MinMaxScaler # para normalização
from sklearn.preprocessing import StandardScaler # para padronização

## transformações categóricas
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [8]:
# Versões das Bibliotecas utilizadas

pd_pd = pd.__version__
np_np = np.__version__
sns_sns = sns.__version__

print('Versões utilizadas na análise:')
print('Pandas = '+pd_pd)
print('Numpy = '+np_np)
print('Seaborn = '+sns_sns)

Versões utilizadas na análise:
Pandas = 1.5.3
Numpy = 1.22.4
Seaborn = 0.12.2


## **3. Carregamento dos dados**

Os dados estão disponíveis no site do Banco Central e são publicados desde 2014. 
O relatório trimestral é divulgado no formato CSV, disponível para download, não sendo possível sua consulta direta no site. 

https://www3.bcb.gov.br/ranking/historico.do


In [9]:
# carregamento do dataset através da url do github

url = 'https://raw.githubusercontent.com/simonefmr/teste/main/ranking_irregularidades.csv'

In [13]:

df = pd.read_csv(url, sep=';', encoding='ISO-8859-1')
df.head()

,Ano,Trimestre,Categoria,Tipo,CNPJ IF,Instituição financeira,Irregularidade,Quantidade de reclamações reguladas procedentes,Quantidade de reclamações reguladas - outras,Quantidade de reclamações não reguladas,Quantidade total de reclamações,Unnamed: 11
0,2023,1º,"Demais Bancos, Financeiras e Instituições de P...",Conglomerado,,ABC-BRASIL (conglomerado),Cobrança irregular de tarifa relacionada a con...,0,1,0,1,NaN
1,2023,1º,"Demais Bancos, Financeiras e Instituições de P...",Conglomerado,,ABC-BRASIL (conglomerado),"Débito em conta não autorizado pelo cliente, o...",0,1,0,1,NaN
2,2023,1º,"Demais Bancos, Financeiras e Instituições de P...",Conglomerado,,ABC-BRASIL (conglomerado),Insatisfação com a resposta recebida da instit...,0,2,0,2,NaN
3,2023,1º,"Demais Bancos, Financeiras e Instituições de P...",Conglomerado,,ABC-BRASIL (conglomerado),Insatisfação com o atendimento prestado pelo S...,0,0,1,1,NaN
4,2023,1º,"Demais Bancos, Financeiras e Instituições de P...",Conglomerado,,ABC-BRASIL (conglomerado),Irregularidades relacionadas ao acompanhamento...,1,0,0,1,NaN


### **3.1. Condições ou restrições impostas para selecionar os dados**

Não houve dificuldade de carregamento dos dados. Foi preciso ajustar o separador e o encoding para correta leitura do dataset.

### **3.2. Definição dos atributos**

In [14]:
# labels dos atributos do dataset

df.columns

Index(['Ano', 'Trimestre', 'Categoria', 'Tipo', 'CNPJ IF',
       'Instituição financeira', 'Irregularidade',
       'Quantidade de reclamações reguladas procedentes',
       'Quantidade de reclamações reguladas - outras',
       'Quantidade de reclamações não reguladas',
       'Quantidade total de reclamações', 'Unnamed: 11'],
      dtype='object')

**Definindo o que cada atributo representa**

**Ano:** Ano de referência.

**Trimestre:** Trimestre de referência

**Categoria Listagem do ranking:** Mais de dois milhões de clientes ou Menos de dois milhões de clientes.

**Tipo:** Tipo de instituição: Bancos e financeiras ou
Conglomerados.

**CNPJ IF:** CNPJ da instituição financeira (para bancos e financeiras). *Para conglomerados, esse campo não é preenchido.*

**Instituição financeira:** Nome da instituição financeira ou do conglomerado.

**Irregularidade:** Nome da irregularidade.

**Quantidade de reclamações reguladas procedentes:** Quantidade de reclamações reguladas procedentes,
associadas à irregularidade, encerradas no mês de
referência.

**Quantidade de reclamações reguladas - outras:** Quantidade de reclamações reguladas – outras, associadas à irregularidade, encerradas no mês de referência (Não procedentes).

**Quantidade de reclamações não reguladas:** Quantidade de reclamações não reguladas, associadas à irregularidade, encerradas no mês de referência (Irreguladas são reclamações que não tem base regulamentada ou que sua regulamentação não é de competência do Banco Central.

**Quantidade total de reclamações:** Quantidade total de reclamações, associadas à irregularidade, encerradas no mês de referência.

## **4. Análise de Dados**

### **4.1. Limpeza preliminar dos dados**

### **4.2. Análise Descritiva**

### **4.3. Tratamento de Missings**

## **5. Visualizações**

## **6. Pré-processamento**

### **6.1. Divisão do dataset em conjunto de treino e teste**

### **6.2. Normalização e padronização**

### **6.3. Transformações categóricas**

## Conclusão**